# Homework №2

## Imports

In [46]:
import dill
import numpy as np
import pandas as pd
from rectools.columns import Columns
from rectools.dataset import Dataset
from rectools.models import PopularModel
from rectools.models.lightfm import LightFMWrapperModel
from typing import Tuple, Dict

## Обработка данных

### Получаем фичи для юзера

In [47]:
def encode_cat_cols(df: pd.DataFrame, cat_cols) -> Tuple[pd.DataFrame, Dict]:
    cat_col_encoding = {}  # словарь с категориями

    # Тут мы могли бы заполнять пропуски как еще одну категорию,
    # но они и так заполняются таким образом автоматически ниже
    # default_values = {col: 'None' for col in cat_cols}
    # df.fillna(default_values, inplace=True)

    for col in cat_cols:
        cat_col = df[col].astype('category').cat
        cat_col_encoding[col] = cat_col.categories
        df[col] = cat_col.codes.astype('category')
    return df, cat_col_encoding

In [48]:
users = pd.read_csv("data/users.csv")
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
...,...,...,...,...,...
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0


In [49]:
users_features = users.copy(deep=True)
users_cat_cols = [
    'age', 'income', 'sex', 'kids_flg'
]
users_features, users_cat_col_encoding = encode_cat_cols(users_features, users_cat_cols)
users_features

,user_id,age,income,sex,kids_flg
0,973171,1,4,1,1
1,962099,0,2,1,0
2,1047345,3,3,0,0
3,721985,3,2,0,0
4,704055,2,4,0,0
...,...,...,...,...,...
840192,339025,5,0,0,0
840193,983617,0,2,0,1
840194,251008,-1,-1,-1,0
840195,590706,-1,-1,0,0


### Получаем фичи для айтемов

In [50]:
items = pd.read_csv("data/items.csv")
items.head(3)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."


In [51]:
items[items["release_year"].isna()]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
89,10950,series,Начни дома,NaN,NaN,фитнес,Россия,NaN,6.0,NaN,NaN,NaN,Приведи себя в хорошую форму дома с минимальны...,"Начни, дома, Россия, спорт-фитнес, спорт-трене..."
294,3901,series,Красота лица,NaN,NaN,фитнес,Россия,NaN,0.0,NaN,NaN,NaN,"Motify научит вас простым, но очень эффективны...","Красота, лица, 2020, Россия, спорт-фитнес, спо..."
374,10656,series,Медитации для жизни,NaN,NaN,фитнес,NaN,NaN,6.0,NaN,NaN,NaN,"В этой программе собраны 3 базовых урока, на к...","Медитации, для, жизни, спорт-фитнес, спорт-тре..."
375,11526,series,Психология питания,NaN,NaN,"фитнес, спорт",NaN,NaN,6.0,NaN,NaN,NaN,В программе от Motify ты узнаешь о психологиче...,"Психология, питания"
572,11532,series,Йога старт,NaN,NaN,фитнес,Россия,NaN,6.0,NaN,NaN,NaN,Программа от Motify подойдет для любого уровня...,"Йога, 2020, Россия, спорт-фитнес, спорт-тренер..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15391,12643,series,Идеальная осанка и мышечный рельеф,NaN,NaN,фитнес,Россия,NaN,6.0,NaN,NaN,NaN,Программа от Motify представляет собой функцио...,"Идеальная, осанка, мышечный, рельеф, Россия, с..."
15609,15173,film,Порт - Сересо Осака,NaN,NaN,футбол,Россия,NaN,0.0,NaN,NaN,NaN,Лига чемпионов AFC - это главный клубный футбо...,"Порт, -, Сересо, Осака, Россия, спорт-фитнес, ..."
15826,8206,series,Рацион на 1478 ккал,NaN,NaN,фитнес,Россия,NaN,6.0,NaN,NaN,NaN,Готовый рацион на день от Motify. Состоит из п...,"Рацион, 1478, ккал, Россия, спорт-фитнес, спор..."
15827,13968,series,Рацион на 2534 ккал,NaN,NaN,фитнес,Россия,NaN,6.0,NaN,NaN,NaN,Готовые рецепты с подробной инструкцией. 4 при...,"Рацион, 2534, ккал, Россия, спорт-фитнес, спор..."


In [52]:
items[items["age_rating"].isna()]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
8297,6316,series,Машины-помощники,NaN,2016.0,"русские, для детей, сериалы, хочу всё знать, р...",Россия,NaN,NaN,NaN,NaN,NaN,Маленькие мальчики всего мира любят играть в м...,"Машины-помощники, 2017, Россия, работа, преодо..."
12352,14409,series,БиБаБу,NaN,2017.0,"русские, сериалы, для детей, хочу всё знать, р...",Россия,NaN,NaN,NaN,NaN,NaN,В сказочное подводное царство приглашают мален...,"друзья, магия и волшебство, рыбы, моря и океан..."


In [53]:
print(len(items["countries"].unique()))
items["countries"].unique()

688


array(['Испания', 'США', 'Канада', 'Великобритания', 'СССР', 'Россия',
       'Иран', 'Германия', 'Италия', 'Аргентина', 'Франция', 'Украина',
       'Россия, Украина', 'США, Германия, Япония, Великобритания',
       'Швеция', 'Норвегия', 'Германия, Канада', 'Россия, Италия, Швеция',
       'Чехия', 'Китай, Япония, Тайвань, Республика Корея', 'Израиль',
       'Великобритания, Франция',
       'Дания, Швеция, США, Великобритания, Аргентина, Германия, Исландия, Испания',
       'Бельгия', 'США, Канада', 'Франция, Бельгия', 'Республика Корея',
       'Казахстан', 'Новая Зеландия', 'Австралия', 'Мексика',
       'Великобритания, США', 'Швеция, Испания, США',
       'США, Великобритания', 'Нидерланды, Великобритания',
       'США, Германия', 'Япония', 'Китай',
       'Великобритания, Швеция, Германия', 'США, Китай, Германия, Япония',
       'Великобритания, США, Китай', 'Кипр', 'Маврикий', 'США, Гонконг',
       'США, Китай, Япония', 'Ирландия', 'Германия, Норвегия',
       'Болгария, Вели

In [54]:
print(len(items["directors"].unique()))
items["directors"].unique()

7974


array(['Педро Альмодовар', 'Скот Армстронг', 'Адам П. Калтраро', ...,
       'Эшреф Рейбрук, Амир Камдин, Эрик Эгер',
       'Марк О’Коннор, Конор МакМахон', 'Михаил Миронов'], dtype=object)

In [55]:
items_features = items.copy(deep=True)
default_values_items = {
    'release_year': items['release_year'].median(),
    'age_rating': items['age_rating'].median(),
}
dropped_columns = [
    "title", "title_orig", "genres", "countries", "directors", "actors", "description", "keywords" 
]
cat_columns = [
    "content_type", "for_kids", "studios"
]
items_features.fillna(default_values_items, inplace=True)
items_features.drop(dropped_columns, axis=1, inplace=True)
items_features, items_cat_col_encoding = encode_cat_cols(items_features, cat_columns)
items_features.head(3)

,item_id,content_type,release_year,for_kids,age_rating,studios
0,10711,0,2002.0,-1,16.0,-1
1,2508,0,2014.0,-1,16.0,-1
2,10716,0,2011.0,-1,16.0,-1


### Получаем фичи интеракций

In [73]:
interactions = pd.read_csv("data/interactions.csv")
interactions["last_watch_dt"] = interactions["last_watch_dt"].astype(np.datetime64)
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


In [80]:
interactions_features = interactions.copy(deep=True)
# Получаем длину истории юзера 
interactions_features['user_hist'] = (
    interactions_features.groupby('user_id')
    ['item_id'].transform('count')
)
# Получаем популярность контента
interactions_features['item_pop'] = (
    interactions_features.groupby('item_id')
    ['user_id'].transform('count')
)
# Получаем среднюю популярность контента, просматриваемого этим юзером
interactions_features['user_avg_pop'] = (
    interactions_features.groupby('user_id')
    ['item_pop'].transform('mean')
)
# Получаем среднюю длину истории пользователя, которые смотрит этот контент
interactions_features['item_avg_hist'] = (
    interactions_features.groupby('item_id')
    ['user_hist'].transform('mean')
)
# Получаем популярность последнего просмотренного контента
interactions_features.sort_values(
    by=[Columns.User, "last_watch_dt"], 
    ascending=[True, False], 
    ignore_index=True,
    inplace=True,
)
interactions_features['user_last_pop'] = (
    interactions_features.groupby('user_id')
    ['item_pop'].transform('first')
)
interactions_features

,user_id,item_id,last_watch_dt,total_dur,watched_pct,user_hist,item_pop,user_avg_pop,item_avg_hist,user_last_pop
0,0,6006,2021-07-20,1,0.0,6,8094,66528.666667,19.439461,8094
1,0,7102,2021-07-19,169,3.0,6,19211,66528.666667,22.676956,8094
2,0,14359,2021-07-19,130,2.0,6,7637,66528.666667,24.844180,8094
3,0,15297,2021-07-19,459,0.0,6,193123,66528.666667,8.728955,8094
4,0,9728,2021-07-19,4,0.0,6,132865,66528.666667,12.240997,8094
...,...,...,...,...,...,...,...,...,...,...
5476246,1097555,4662,2021-04-08,775,14.0,7,401,30366.571429,56.980050,132865
5476247,1097556,12812,2021-07-11,1417,23.0,1,1812,1812.000000,29.582230,1812
5476248,1097557,4151,2021-07-26,638,13.0,3,91167,98713.666667,12.420722,91167
5476249,1097557,3182,2021-07-26,505,8.0,3,11851,98713.666667,24.163952,91167


In [84]:
items_features_enhanced = pd.merge(
    left=items_features, 
    right=(
        interactions_features
        [['item_id', 'item_pop', 'item_avg_hist']]
        .drop_duplicates()
    ),
    how='left',
    on='item_id',
)
items_features_enhanced

,item_id,content_type,release_year,for_kids,age_rating,studios,item_pop,item_avg_hist
0,10711,0,2002.0,-1,16.0,-1,5.0,108.200000
1,2508,0,2014.0,-1,16.0,-1,9.0,76.444444
2,10716,0,2011.0,-1,16.0,-1,6.0,68.500000
3,7868,0,2015.0,-1,16.0,-1,2.0,100.000000
4,16268,0,1978.0,-1,12.0,33,1.0,36.000000
...,...,...,...,...,...,...,...,...
15958,6443,1,2018.0,-1,16.0,-1,7668.0,23.085811
15959,2367,1,2020.0,0,18.0,-1,396.0,29.573232
15960,10632,1,2017.0,0,18.0,-1,57.0,54.701754
15961,4538,1,2019.0,0,18.0,-1,99.0,29.111111


In [85]:
users_features_enhanced = pd.merge(
    left=users, 
    right=(
        interactions_features
        [['user_id', 'user_hist', 'user_avg_pop', 'user_last_pop']]
        .drop_duplicates()
    ),
    how='left',
    on='user_id',
)
users_features_enhanced

,user_id,age,income,sex,kids_flg,user_hist,user_avg_pop,user_last_pop
0,973171,age_25_34,income_60_90,М,1,5.0,25637.800000,122119.0
1,962099,age_18_24,income_20_40,М,0,14.0,1801.785714,643.0
2,1047345,age_45_54,income_40_60,Ж,0,NaN,NaN,NaN
3,721985,age_45_54,income_20_40,Ж,0,18.0,7703.111111,4416.0
4,704055,age_35_44,income_60_90,Ж,0,10.0,590.900000,983.0
...,...,...,...,...,...,...,...,...
840192,339025,age_65_inf,income_0_20,Ж,0,1.0,1976.000000,1976.0
840193,983617,age_18_24,income_20_40,Ж,1,NaN,NaN,NaN
840194,251008,NaN,NaN,NaN,0,2.0,70.000000,105.0
840195,590706,NaN,NaN,Ж,0,1.0,359.000000,359.0


In [83]:
default_values_items['item_pop'] = interactions_features['item_pop'].median()
default_values_items['item_avg_hist'] = interactions_features['item_avg_hist'].median()

# Для новых фичей юзеров
default_values_users = {
    'user_hist': 0,
    'user_avg_pop': interactions_features['user_avg_pop'].median(),
    'user_last_pop': interactions_features['user_last_pop'].median(),
}
default_values_users

{'user_hist': 0, 'user_avg_pop': 17007.285714285714, 'user_last_pop': 3820.0}

In [86]:
interactions_default_values = {
    "last_watch_dt": interactions["last_watch_dt"].median(),
    Columns.Weight: 1,
    "watched_pct": 0,
    'total_dur': 0,
}
interactions_default_values

{'last_watch_dt': Timestamp('2021-07-01 00:00:00'),
 'weight': 1,
 'watched_pct': 0,
 'total_dur': 0}

## Генерим кандидатов для ранкера

Используем две модели первого уровня для генерации:
- Обученную LightFM из прошлой дз
- Популярную модель

In [87]:
with open(f"dataset.dill", "rb") as file:
    dataset: Dataset = dill.load(file)
train_interactions = dataset.interactions.df
train_interactions

,user_id,item_id,weight,last_watch_dt
0,0,0,3.0,2021-05-11
1,1,1,3.0,2021-05-29
3,2,2,3.0,2021-07-05
4,3,0,3.0,2021-04-30
5,4,3,3.0,2021-05-13
...,...,...,...,...
5476242,233267,46,3.0,2021-04-21
5476244,54448,166,3.0,2021-08-02
5476245,173679,100,1.0,2021-05-12
5476247,165659,2141,3.0,2021-04-13


In [88]:
with open(f"test_interactions.dill", "rb") as file:
    test_interactions: pd.DataFrame = dill.load(file)
test_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
9,203219,13582,2021-08-22,6975,100.0,3
54,200197,9335,2021-08-16,83,2.0,1
64,73446,14488,2021-08-19,6011,100.0,3
84,10010,512,2021-08-15,14,0.0,1
94,890735,14200,2021-08-16,1179,28.0,3
...,...,...,...,...,...,...
5476169,589589,983,2021-08-21,2403,43.0,3
5476188,590892,8618,2021-08-21,1335,23.0,3
5476191,857162,12360,2021-08-16,11,0.0,1
5476201,273558,10605,2021-08-21,34030,100.0,3


In [60]:
TEST_USERS = test_interactions[Columns.User].unique()
TEST_USERS

array([203219, 200197,  73446, ..., 623792, 442859, 857162], dtype=int64)

In [61]:
with open(f"LightFM_warp_8_0784.dill", "rb") as file:
    lightfm_model: LightFMWrapperModel = dill.load(file)
print(lightfm_model)

In [89]:
n_candidates = 100 # Число кандидатов (от каждой модели?)

In [63]:
lightfm_recos = lightfm_model.recommend(users=TEST_USERS, dataset=dataset, k=n_candidates, filter_viewed=True)
lightfm_recos.head()

,user_id,item_id,score,rank
0,203219,15297,3.580714,1
1,203219,10440,3.536684,2
2,203219,13865,3.146872,3
3,203219,4151,3.101431,4
4,203219,2657,3.067135,5


In [64]:
popular_model = PopularModel()
popular_model.fit(dataset);

In [65]:
popular_recos = popular_model.recommend(users=TEST_USERS, dataset=dataset, k=n_candidates, filter_viewed=True)
popular_recos.head()

,user_id,item_id,score,rank
0,203219,10440,159230.0,1
1,203219,15297,158315.0,2
2,203219,13865,96838.0,3
3,203219,9728,92359.0,4
4,203219,4151,70650.0,5


Комбинируем рекомендации обоих моделей

In [66]:
combined_recos = pd.merge(
    lightfm_recos, popular_recos, on=["user_id", "item_id"], how="outer"
)
combined_recos.sort_values(["user_id",	"item_id"], inplace=True)
combined_recos

,user_id,item_id,score_x,rank_x,score_y,rank_y
5827259,1,14,-227.010579,60.0,5667.0,73.0
13091046,1,24,NaN,NaN,4861.0,92.0
5827252,1,101,-226.932782,53.0,6661.0,58.0
5827206,1,142,-225.937359,7.0,35213.0,8.0
5827261,1,657,-227.025621,62.0,6490.0,59.0
...,...,...,...,...,...,...
12102109,1097544,16270,NaN,NaN,5780.0,69.0
2592690,1097544,16291,-228.083697,91.0,NaN,NaN
2592657,1097544,16361,-227.835225,58.0,5336.0,80.0
2592689,1097544,16447,-228.083070,90.0,NaN,NaN


In [67]:
# combined_recos_filled = combined_recos.copy(deep=True)
# for prefix in ["x", "y"]:
#     combined_recos_filled[f"score_{prefix}"] = combined_recos.groupby("user_id")[f"score_{prefix}"].apply(lambda x: x.fillna(x.min() - 0.01))
#     combined_recos_filled[f"rank_{prefix}"] = combined_recos.groupby("user_id")[f"rank_{prefix}"].apply(lambda x: x.fillna(x.max() + 1))
# combined_recos_filled

## Объединяем кандидатов с интеракциями, а также добавляем все фичи (юзеров, айтемов, интеракций)

Объединяем итоговых кандидатов с реальными интеракциями пользователей из теста

In [68]:
df = pd.merge(
    test_interactions,
    combined_recos, # combined_recos_filled, # lightfm_recos, 
    how='outer',  # right ? 
    on=['user_id', 'item_id']
)

# # Проставляем дефолтные значения интеракций
# min_score: float =  df['score'].min() - 0.01
# max_rank: int = df['rank'].max() + 1  # 101

for prefix in ["x", "y"]:
    df[f"score_{prefix}"] = df.groupby("user_id")[f"score_{prefix}"].apply(lambda x: x.fillna(x.min() - 0.01))
    df[f"rank_{prefix}"] = df.groupby("user_id")[f"rank_{prefix}"].apply(lambda x: x.fillna(x.max() + 1))

default_values = {
    # 'score': min_score, 'rank': max_rank,
    # Важно использовате те же дефолтные значения для интеракций, 
    # чтобы не сделать утечку
    **interactions_default_values,
}
df.fillna(default_values, inplace=True)
df

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight,score_x,rank_x,score_y,rank_y
0,203219,13582,2021-08-22,6975.0,100.0,3.0,1.721243,101.0,4503.99,101.0
1,200197,9335,2021-08-16,83.0,2.0,1.0,-226.284075,101.0,4503.99,101.0
2,73446,14488,2021-08-19,6011.0,100.0,3.0,-206.203359,101.0,4088.99,101.0
3,10010,512,2021-08-15,14.0,0.0,1.0,-205.246856,101.0,4432.99,101.0
4,890735,14200,2021-08-16,1179.0,28.0,3.0,-160.690570,101.0,4503.99,101.0
...,...,...,...,...,...,...,...,...,...,...
14942813,1097544,16270,2021-07-01,0.0,0.0,1.0,-228.128515,101.0,5780.00,69.0
14942814,1097544,16291,2021-07-01,0.0,0.0,1.0,-228.083697,91.0,4503.99,101.0
14942815,1097544,16361,2021-07-01,0.0,0.0,1.0,-227.835225,58.0,5336.00,80.0
14942816,1097544,16447,2021-07-01,0.0,0.0,1.0,-228.083070,90.0,4503.99,101.0
